# 03 — Derivadas de MSE y Logística + validación con autograd (PyTorch)

In [ ]:

# %pip install numpy matplotlib torch scikit-learn --quiet
import numpy as np, torch, matplotlib.pyplot as plt
from sklearn.datasets import make_regression, make_classification
np.random.seed(0); torch.manual_seed(0)


In [ ]:

# Regresión: gradiente MSE analítico vs autograd
X, y = make_regression(n_samples=400, n_features=5, noise=10, random_state=0)
X = np.c_[np.ones(len(X)), X]
w = np.zeros(X.shape[1])
def mse_grad(X, y, w):
    n = len(y)
    return (2/n) * X.T @ (X @ w - y)
print("||grad MSE analítico||:", np.linalg.norm(mse_grad(X,y,w)))

Xt = torch.tensor(X, dtype=torch.float32)
yt = torch.tensor(y, dtype=torch.float32).view(-1,1)
wt = torch.zeros((X.shape[1],1), requires_grad=True)
loss = torch.mean((Xt@wt - yt)**2)
loss.backward()
print("||grad MSE autograd||:", torch.norm(wt.grad).item())


In [ ]:

# Clasificación logística: gradiente y descenso de gradiente
Xc, yc = make_classification(n_samples=500, n_features=4, n_informative=3, random_state=0)
Xc = np.c_[np.ones(len(Xc)), Xc]
def sigmoid(z): return 1/(1+np.exp(-z))
def logloss_grad(X,y,w):
    p = sigmoid(X@w)
    g = X.T @ (p - y) / len(y)
    L = -np.mean(y*np.log(p+1e-9)+(1-y)*np.log(1-p+1e-9))
    return L, g
w = np.zeros(Xc.shape[1])
lr=0.5; losses=[]
for _ in range(200):
    L,g = logloss_grad(Xc, yc, w); w -= lr*g; losses.append(L)
plt.figure(figsize=(5,4)); plt.plot(losses); plt.title("Log-loss"); plt.xlabel("iter"); plt.ylabel("loss"); plt.show()
